#Web Scraping em Python

In [36]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

###O primeiro passo será fazer uma requisição GET para a página. Para isso, devemos chamar o método requests.get com a URL da página como argumento.

###Este método retorna um objeto Response que contém vários informações, das quais utilizaremos o status_code para verificar se a requisição retornou um status 200, indicando que ela foi bem sucedida, e content para acessar o código da página HTML.

In [37]:
req = requests.get('https://www.basketball-reference.com/leagues/NBA_2018_totals.html')
if req.status_code == 200:
    print('Requisição bem sucedida!')
    content = req.content

Requisição bem sucedida!


###Depois de obter o HTML da página, podemos utilizar a biblioteca BeautifulSoup para extrair a tabela. Primeiro, devemos criar um objeto que irá salvar o documento de maneira estruturada de acordo com as tags, e depois podemos acessar o elemento que quisermos chamando o método find passando como argumento o nome da tag, no caso table.

In [38]:
soup = BeautifulSoup(content, 'html.parser')
table = soup.find(name='table')

###Agora que temos o código HTML da tabela, podemos utilizar o Pandas para carregar os dados em um Data Frame, utilizando o método read_html. Para isto, existem dois pontos para ficar atento: o primeiro é que antes de passar a variável ‘table’ na função, devemos convertê-la para string primeiro, tendo em vista que no momento ela é um objeto do BeautifulSoup; o segundo é que o retorno deste método é sempre uma lista de Data Frames, e portanto devemos acessar a posição 0 dela para obter nossa tabela.

In [39]:
table_str = str(table)
df = pd.read_html(table_str)[0]


In [40]:
df.head()

,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,1,Álex Abrines,SG,24,OKC,75,8,1134,115,291,.395,84,221,.380,31,70,.443,.540,39,46,.848,26,88,114,28,38,8,25,124,353
1,2,Quincy Acy,PF,27,BRK,70,8,1359,130,365,.356,102,292,.349,28,73,.384,.496,49,60,.817,40,217,257,57,33,29,60,149,411
2,3,Steven Adams,C,24,OKC,76,76,2487,448,712,.629,0,2,.000,448,710,.631,.629,160,286,.559,384,301,685,88,92,78,128,215,1056
3,4,Bam Adebayo,C,20,MIA,69,19,1368,174,340,.512,0,7,.000,174,333,.523,.512,129,179,.721,118,263,381,101,32,41,66,138,477
4,5,Arron Afflalo,SG,32,ORL,53,3,682,65,162,.401,27,70,.386,38,92,.413,.485,22,26,.846,4,62,66,30,4,9,21,56,179


###E se a página tiver mais de uma tabela?
###No caso anterior, a página continha somente uma tabela, então havia somente uma tag table para extrair. No entanto, em muitas páginas existem várias tabelas, como é o caso da página de Classificações da temporada de 2017/2018 da NBA, por exemplo. Neste caso, para obter uma tabela específica existem duas opções:
###A primeira é substituir o método find pelo find_all, que retorna uma lista de todos os elementos encontrados ao invés de um só. Nesse caso, é possível acessar a tabela desejada verificando em qual posição do vetor ela se encontra.
###A segunda é utilizar o argumento attrs do método find, passando um dicionário que indica quais atributos o elemento obrigatoriamente deve ter para ser extraído. Por exemplo, considerando a página de classificações citada acima e que queremos extrair as colocações dos times na conferência Oeste (Western Conference), usamos o inspetor para verificar que o id dessa tabela é “confs_standings_W”. Portanto, o código ficaria da seguinte forma:

In [41]:
table = soup.find(name='table', attrs={'id':'confs_standings_W'})

###Obtendo estatísticas de várias temporadas
###Agora que conseguimos extrair dados de uma única página, seria interessante obter dados de várias temporadas de uma só vez. Comparando a URL de Estatísticas de 2018 com a de 2017, podemos ver que elas são iguais, com exceção do número do ano da temporada.
###Com isso, é possível criar um loop que itere sobre uma lista de anos incluindo eles na URL e repetindo o processo da seção anterior para cada uma delas, montando uma grande tabela. Para obter uma lista de todos os anos em um certo intervalo, podemos usar a função range nativa do Python.
###O código a seguir cria uma função que faz isso automaticamente, e usa ela para extrair as estatísticas totais de 2013 a 2018. Perceba que uma coluna Year é criada em cada extração para que seja possível diferenciar de qual ano cada estatística pertence no DataFrame principal.

In [42]:
def scrape_stats(base_url, year_start, year_end):
    years = range(year_start,year_end+1,1)

    final_df = pd.DataFrame()

    for year in years:
        print('Extraindo ano {}'.format(year))
        req_url = base_url.format(year)
        req = requests.get(req_url)
        soup = BeautifulSoup(req.content, 'html.parser')
        table = soup.find('table', {'id':'totals_stats'})
        df = pd.read_html(str(table))[0]
        df['Year'] = year
        final_df = final_df.append(df)
    return final_df
url = 'https://www.basketball-reference.com/leagues/NBA_{}_totals.html'
df = scrape_stats(url, 2013, 2018)

Extraindo ano 2013
Extraindo ano 2014
Extraindo ano 2015
Extraindo ano 2016
Extraindo ano 2017
Extraindo ano 2018


In [43]:
df

,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Year
0,1,Quincy Acy,PF,22,TOR,29,0,342,42,75,.560,1,2,.500,41,73,.562,.567,31,38,.816,30,47,77,11,13,15,17,53,116,2013
1,2,Jeff Adrien,PF,26,CHA,52,5,713,72,168,.429,0,2,.000,72,166,.434,.429,65,100,.650,68,128,196,36,18,27,32,80,209,2013
2,3,Arron Afflalo,SF,27,ORL,64,64,2307,397,905,.439,72,240,.300,325,665,.489,.478,191,223,.857,29,210,239,206,40,11,138,137,1057,2013
3,4,Josh Akognon,PG,26,DAL,3,0,9,2,4,.500,1,2,.500,1,2,.500,.625,0,0,NaN,0,1,1,1,0,0,0,3,5,2013
4,5,Cole Aldrich,C,24,TOT,45,0,388,44,80,.550,0,0,NaN,44,80,.550,.550,12,20,.600,30,90,120,9,5,23,23,60,100,2013
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
685,537,Tyler Zeller,C,28,BRK,42,33,703,125,229,.546,10,26,.385,115,203,.567,.568,40,60,.667,63,131,194,28,8,21,35,78,300,2018
686,537,Tyler Zeller,C,28,MIL,24,1,406,62,105,.590,0,2,.000,62,103,.602,.590,17,19,.895,47,64,111,19,7,14,12,48,141,2018
687,538,Paul Zipser,SF,23,CHI,54,12,824,81,234,.346,37,110,.336,44,124,.355,.425,19,25,.760,13,118,131,46,20,15,43,86,218,2018
688,539,Ante Žižić,C,21,CLE,32,2,214,49,67,.731,0,0,NaN,49,67,.731,.731,21,29,.724,24,36,60,5,2,13,11,30,119,2018


###Pequeno exemplo de uso dos dados
###Primeiro, algo importante a se fazer é fazer uma pequena limpeza dos dados. Olhando a tabela no site, podemos ver que os nomes das colunas se repetem várias vezes no meio dela. Podemos eliminar essas linhas do Data Frame da seguinte maneira:

In [44]:
drop_indexes = df[df['Rk'] == 'Rk'].index # Pega indexes onde a coluna 'Rk' possui valor 'Rk'
df.drop(drop_indexes, inplace=True) # elimina os valores dos index passados da tabela

In [48]:
df

,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Year
0,1,Quincy Acy,PF,22,TOR,29,0,342,42,75,0.560,1,2,0.500,41,73,0.562,0.567,31,38,0.816,30,47,77,11,13,15,17,53,116,2013
1,2,Jeff Adrien,PF,26,CHA,52,5,713,72,168,0.429,0,2,0.000,72,166,0.434,0.429,65,100,0.650,68,128,196,36,18,27,32,80,209,2013
2,3,Arron Afflalo,SF,27,ORL,64,64,2307,397,905,0.439,72,240,0.300,325,665,0.489,0.478,191,223,0.857,29,210,239,206,40,11,138,137,1057,2013
3,4,Josh Akognon,PG,26,DAL,3,0,9,2,4,0.500,1,2,0.500,1,2,0.500,0.625,0,0,NaN,0,1,1,1,0,0,0,3,5,2013
4,5,Cole Aldrich,C,24,TOT,45,0,388,44,80,0.550,0,0,NaN,44,80,0.550,0.550,12,20,0.600,30,90,120,9,5,23,23,60,100,2013
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
685,537,Tyler Zeller,C,28,BRK,42,33,703,125,229,0.546,10,26,0.385,115,203,0.567,0.568,40,60,0.667,63,131,194,28,8,21,35,78,300,2018
686,537,Tyler Zeller,C,28,MIL,24,1,406,62,105,0.590,0,2,0.000,62,103,0.602,0.590,17,19,0.895,47,64,111,19,7,14,12,48,141,2018
687,538,Paul Zipser,SF,23,CHI,54,12,824,81,234,0.346,37,110,0.336,44,124,0.355,0.425,19,25,0.760,13,118,131,46,20,15,43,86,218,2018
688,539,Ante Žižić,C,21,CLE,32,2,214,49,67,0.731,0,0,NaN,49,67,0.731,0.731,21,29,0.724,24,36,60,5,2,13,11,30,119,2018


###Outra coisa que tem que ser feita é converter os valores que representam números na tabela, pois quando o Panda pega a tabela do HTML, todos os dados são lidos como objetos.

In [49]:
numeric_cols = df.columns.drop(['Player','Pos','Tm'])
df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric)

###Perceba como o número de bolas de 3 arremessadas aumentou consideravelmente nos últimos anos, indo de aproximadamente 90 para 120 entre 2013 e 2017, aonde atingiu uma estabilidade. Podemos concluir que nos últimos anos esse tipo de arremesso passou a ter cada vez mais importância no jogo.
###Já que o volume de bolas de 3 aumentou consideravelmente, quais são os jogadores que mais acertaram nesse período? Vamos verificar o top 5 de arremessadores com manipulações do Pandas.

In [50]:
sorted_df = df.sort_values(by=['3P'],axis=0,ascending=False)
sorted_df[['Player','3P','Year']].head()

,Player,3P,Year
121,Stephen Curry,402,2016
124,Stephen Curry,324,2017
140,Stephen Curry,286,2015
123,Stephen Curry,272,2013
543,Klay Thompson,268,2017
